In [1]:
from BackgammonModel import *

In [ ]:
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)

In [ ]:
model2 = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_333000.pt',
    eval_interval = 1000
)

In [ ]:
env = Env()
model.evaluate(env, {Player.LIGHT: RandomAgent(player = Player.LIGHT), 
                     Player.DARK: RandomAgent(player = Player.DARK)})

In [ ]:
env = Env()
model.evaluate(env, {Player.LIGHT: TDAgent(player = Player.LIGHT, net = model2, env = env),
Player.DARK: TDAgent(player = Player.DARK, net = model, env = env)})

In [ ]:
env = Env()
model.evaluate(env, {Player.LIGHT: TDAgent(player = Player.LIGHT, net = model, env = env),
Player.DARK: TDAgent(player = Player.DARK, net = model2, env = env)})

In [ ]:
env = Env()
model.evaluate(env, {Player.LIGHT: RandomAgent(player = Player.LIGHT),
Player.DARK: TDAgent(player = Player.DARK, net = model2, env = env)})

In [ ]:
model.train(358001,200000)

In [2]:
%load_ext line_profiler
model = BackgammonModel( \
    input_size = 198,
    hidden_size = 50,
    output_size = 1,
    trace_decay = 0.7,
    alpha = 0.1,
    save_interval = 1000,
    saved_model = './checkpoints/50/checkpoint_358000.pt',
    eval_interval = 100
)
env = Env()
agents = {Player.LIGHT: RandomAgent(player = Player.LIGHT),
Player.DARK: TDAgent(player = Player.DARK, net = model, env = env)}

Loading saved model: ./checkpoints/50/checkpoint_358000.pt


In [3]:
%lprun -f model.evaluate model.evaluate(env, agents, num_episodes = 1)


100%|██████████| 1/1 [00:00<00:00,  1.77it/s]


In [4]:
from Backgammon import *
game = Backgammon(verbose=False, max_turns = 200)


In [5]:
%lprun -f game.get_legal_actions game.get_legal_actions()


In [6]:
%lprun -f game.legal_checker_moves_outer game.legal_checker_moves_outer(Player.LIGHT, game.dice)

In [7]:
%lprun -f game.legal_checker_moves game.legal_checker_moves(game.cur_player, game.dice)

In [8]:
a = game.legal_checker_moves(game.cur_player, game.dice)[0]
a

Move(player=<Player.DARK: 1>, pos=5, num=4, hit=False)

In [12]:
%lprun -f game.apply_move game.apply_move(a)

In [11]:
%lprun -f game.update_board_pos game.update_board_pos(15, Player.DARK, 1)